In [138]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [158]:

import math
import time
from tqdm import tqdm



import torch
import numpy as np
import torch.nn as nn
from torch import Tensor

from torch.utils.data import DataLoader
from torch.nn import (TransformerEncoder, TransformerDecoder,
                      TransformerEncoderLayer, TransformerDecoderLayer)

from src.prepare_data import download_data, build_train_vocab, get_train_test_val, check_tokens,tokens_to_sentence , generate_batch , visualize_iter_data , get_embed
from src.LSTM import RNNdecoder, RNNencoder,Seq2SeqRNN
from src.train import create_mask,generate_square_subsequent_mask, train_epoch, bleu_calculate , evaluate
from src.transformer import Seq2SeqTransformer,PositionalEncoding,TokenEmbedding


In [ ]:
# !pip freeze > requirements.txt
# !python3 -m spacy info # spacy работает на версиях питона от 3.10
# !python -m spacy download de_core_news_sm
# !python -m spacy download en_core_web_sm

In [3]:
train_filepaths , val_filepaths , test_filepaths = download_data()
de_vocab, en_vocab, de_tokenizer, en_tokenizer = build_train_vocab(train_filepaths)
print( 'De vocab En vocab: ',len(de_vocab), len(en_vocab))
train_data , val_data , test_data = get_train_test_val(train_filepaths, test_filepaths, val_filepaths , de_vocab , en_vocab ,de_tokenizer,en_tokenizer )
print('Train Test Val: ',len(train_data),len(test_data) , len(val_data))

De vocab En vocab:  19215 10838
Train Test Val:  29000 1014 1000


In [4]:
BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']
print(PAD_IDX , BOS_IDX , EOS_IDX)

1 2 3


In [5]:
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn= lambda x : generate_batch(x , BOS_IDX=BOS_IDX,PAD_IDX=PAD_IDX,EOS_IDX=EOS_IDX))
valid_iter = DataLoader(val_data, batch_size=1,
                        shuffle=True, collate_fn= lambda x : generate_batch(x , BOS_IDX=BOS_IDX,PAD_IDX=PAD_IDX,EOS_IDX=EOS_IDX))
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn= lambda x : generate_batch(x , BOS_IDX=BOS_IDX,PAD_IDX=PAD_IDX,EOS_IDX=EOS_IDX))

In [6]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cpu'

# TEST TRANSFORMER

In [187]:
SRC_VOCAB_SIZE = len(de_vocab)
TGT_VOCAB_SIZE = len(en_vocab)
EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 32
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(num_encoder_layers=NUM_ENCODER_LAYERS,
                                num_decoder_layers= NUM_DECODER_LAYERS,
                                emb_size= EMB_SIZE, src_vocab_size= SRC_VOCAB_SIZE,
                                 tgt_vocab_size= TGT_VOCAB_SIZE,
                                 dim_feedforward= FFN_HID_DIM , NHEAD=NHEAD)
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)

/home/igoreshka/miniconda3/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [188]:
for epoch in range(1, NUM_EPOCHS+1):
    start_time = time.time()
    print('train')
    train_loss = train_epoch(transformer, train_iter, optimizer , DEVICE =DEVICE , loss_fn =loss_fn , pad_idx = PAD_IDX)
    end_time = time.time()
    print('eval')
    val_loss = evaluate(transformer, valid_iter , DEVICE =DEVICE , loss_fn =loss_fn,  pad_idx = PAD_IDX)
    print('bleu')
    bleu = bleu_calculate(transformer, valid_iter, en_vocab = en_vocab ,de_vocab = de_vocab ,de_tokenizer = de_tokenizer ,DEVICE = DEVICE , EOS_IDX = EOS_IDX ,BOS_IDX = BOS_IDX, transformer=True)
    all_time = time.time()
    print(f"Epoch: {epoch}, "
          f"Train loss: {train_loss:.3f}, "
          f"Val loss: {val_loss:.3f}, "
          f"Blue: {bleu:.3f}, "
          f"Epoch time = {(end_time - start_time):.3f}s, "
          f"All time = {(all_time - start_time):.3f}s")

train
eval
bleu
Epoch: 1, Train loss: 0.279, Val loss: 0.061, Blue: 0.000, Epoch time = 19.858s, All time = 21.949s
train
eval
bleu
Epoch: 2, Train loss: 0.268, Val loss: 0.060, Blue: 0.000, Epoch time = 32.781s, All time = 34.947s
train


KeyboardInterrupt: 

In [ ]:
for idx, (src, tgt) in enumerate(train_iter):
        #FORWARD
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input, pad_idx = PAD_IDX, device=DEVICE)
        print('Батч предложений deutch: ',src.shape)
        print('Батч переводов english: ',tgt_input.shape)
        print('Маска предложений входных: ', src_mask.shape)
        print('Маска переводов: ', tgt_mask.shape )
        print('Падинг маска предложений входных: ',src_padding_mask.shape)
        print('Падинг маска переводов: ',tgt_padding_mask.shape)
        logits = transformer.forward(src,
                       tgt_input,
                       src_mask,
                       tgt_mask,
                       src_padding_mask,
                       tgt_padding_mask,
                       src_padding_mask)
        print('Предсказания вероятностей слов на каждой позиции: ',logits.shape)

        #DECODER

        break

# TEST RNN ENCODER DECODER

In [8]:
SRC_VOCAB_SIZE = len(de_vocab)
TGT_VOCAB_SIZE = len(en_vocab)
EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 32
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2


In [9]:

data_object = get_embed(train_iter = train_iter , EMB_SIZE = EMB_SIZE , DEVICE = DEVICE , SRC_VOCAB_SIZE = SRC_VOCAB_SIZE , PAD_IDX = PAD_IDX )
src = data_object['src']
tgt = data_object['tgt']
src_mask = data_object['src_mask']
tgt_mask = data_object['tgt_mask']
src_padding_mask = data_object['src_padding_mask']
tgt_padding_mask = data_object['tgt_padding_mask']
src_emb = data_object['src_emb']
tgt_emb = data_object['tgt_emb']



In [153]:
# src.permute(1,0)[0]
# PositionalEncoding(EMB_SIZE, dropout=0.1)(TokenEmbedding(SRC_VOCAB_SIZE, EMB_SIZE)(src)).permute(1,0,2)[0][:,5]
# src_padding_mask[0]
# src.permute(1,0)[src_padding_mask].shape
# src.permute(1,0)[0][~src_padding_mask[0]]
src_emb.shape , src_mask.shape , src_padding_mask.shape , tgt_emb.shape , tgt_mask.shape , tgt_padding_mask.shape
# torch.sum(src_mask , dim=1) ,torch.sum(src_padding_mask , dim=1)  # src_mask из FALSE вся, но src_padding_mask указывает на паддинговые элементы
# memory[:,-1:,:] == hidden[0].permute(1,0,2)
# tgt_emb.shape , memory.shape , hidden[0].shape , hidden[1].shape

(torch.Size([27, 128, 256]),
 torch.Size([27, 27]),
 torch.Size([128, 27]),
 torch.Size([26, 128, 256]),
 torch.Size([26, 26]),
 torch.Size([128, 26]))

## encoder

In [190]:
torch.sum(src_padding_mask, dim=1)

tensor([ 8, 17, 16, 13, 13, 11,  9,  8, 13, 17, 16, 12,  7, 11, 18, 18, 11, 15,
        13, 15,  6, 15, 13, 17, 12, 10, 16, 14, 12,  9, 13, 17, 16, 15,  7, 15,
         7, 16,  5, 14, 14, 12, 16, 13, 13, 17, 12, 14, 11, 12, 11, 12,  9, 13,
        19, 10, 17,  8, 16, 17, 17, 11, 13,  6, 14, 13,  5, 14,  7, 14, 12, 17,
        10, 10, 11, 16, 10, 11,  7, 10, 17, 13, 15, 16, 12, 13, 11, 16,  7, 12,
        16, 14, 17, 18, 15, 12, 15, 18,  9,  3, 13, 16, 15,  7, 17, 13,  9, 13,
        16, 16,  0,  4, 15, 14, 17, 14,  9, 13, 21, 16,  6,  8, 13, 10, 12, 12,
        14, 16])

In [110]:
encoder = RNNencoder(EMB_SIZE,EMB_SIZE )
memory, hidden = encoder(src_emb , src_padding_mask)
print(memory.shape , hidden[0].shape , hidden[1].shape)

torch.Size([128, 27, 256]) torch.Size([1, 128, 256]) torch.Size([1, 128, 256])


## decoder

In [87]:
lengths = (~tgt_padding_mask).sum(1).int()
lengths

tensor([16,  9, 11, 15, 16, 13, 19, 14, 12, 11, 13, 18, 19, 15, 12, 11, 13, 11,
        14, 12, 20, 13, 15, 10, 17, 17, 11, 11, 15, 14, 13, 14, 13, 18, 23, 15,
        23, 10, 19,  9, 13, 16, 15, 15, 13, 12, 17, 13, 16, 16, 16, 15, 16, 16,
        13, 19, 10, 26, 14, 10, 11, 18, 15, 21, 11, 14, 22, 13, 24, 15, 17, 10,
        17, 15, 13, 13, 17, 13, 21, 17, 13, 14, 12, 12, 19, 17, 18, 12, 18, 14,
        13, 14, 10, 10, 15, 15, 12, 11, 20, 25, 16, 14, 12, 24, 10, 16, 18, 18,
        14, 14, 26, 23, 12, 16, 15, 12, 19, 13,  8, 13, 23, 16, 20, 19, 12, 14,
        13, 11], dtype=torch.int32)

In [88]:
tgt_emb.shape

torch.Size([26, 128, 256])

In [96]:

input = tgt_emb 
result = []
lstm = torch.nn.LSTM(EMB_SIZE, EMB_SIZE, batch_first=True)

result,_ = lstm(input.permute(1,0,2))

# for i in range(input.shape[0]) :
#     print(act.shape)
#     act = input[i:i+1,:,:]
#     act = act.permute(1,0,2)
#     act, hidden = lstm(act,  (hidden[0] , hidden[1]))
#     result.append(act)

# result = torch.stack(result).squeeze(2)
print(result.shape)

torch.Size([128, 26, 256])


In [112]:
decoder = RNNdecoder(EMB_SIZE , EMB_SIZE )
decode_output  = decoder.forward(input = tgt_emb , memory= memory ,hidden=hidden , tgt_padding_mask = tgt_padding_mask )
print(decode_output.shape)


torch.Size([128, 26, 256])


# seq2seq

In [ ]:
%load_ext autoreload
%autoreload 2

In [159]:
SRC_VOCAB_SIZE = len(de_vocab)
TGT_VOCAB_SIZE = len(en_vocab)
EMB_SIZE = 64
NHEAD = 8
FFN_HID_DIM = 64
BATCH_SIZE = 8
NUM_EPOCHS = 10

rnn = Seq2SeqRNN(emb_size= EMB_SIZE, 
                src_vocab_size= SRC_VOCAB_SIZE,
                tgt_vocab_size= TGT_VOCAB_SIZE,
                hidden_size= FFN_HID_DIM)
for p in rnn.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
rnn = rnn.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    rnn.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)

In [160]:
out_forward = rnn.forward(src,tgt, src_mask , tgt_mask , src_padding_mask, tgt_padding_mask , src_padding_mask)
print(out_forward.shape)

torch.Size([128, 26, 10838])


In [185]:
for epoch in range(1, NUM_EPOCHS+1):
    start_time = time.time()
    print('train')
    train_loss = train_epoch(rnn, train_iter, optimizer , DEVICE =DEVICE , loss_fn =loss_fn , pad_idx = PAD_IDX)
    end_time = time.time()
    print('eval')
    val_loss = evaluate(rnn, valid_iter , DEVICE =DEVICE , loss_fn =loss_fn,  pad_idx = PAD_IDX)
    print('bleu')
    bleu = bleu_calculate(rnn, valid_iter, en_vocab = en_vocab ,de_vocab = de_vocab ,de_tokenizer = de_tokenizer ,DEVICE = DEVICE , EOS_IDX = EOS_IDX ,BOS_IDX = BOS_IDX, transformer=False)
    all_time = time.time()
    print(f"Epoch: {epoch}, "
          f"Train loss: {train_loss:.3f}, "
          f"Val loss: {val_loss:.3f}, "
          f"Blue: {bleu:.3f}, "
          f"Epoch time = {(end_time - start_time):.3f}s, "
          f"All time = {(all_time - start_time):.3f}s")

train
eval
bleu
Epoch: 1, Train loss: 0.285, Val loss: 0.064, Blue: 0.002, Epoch time = 8.194s, All time = 8.626s
train
eval
bleu
Epoch: 2, Train loss: 0.285, Val loss: 0.064, Blue: 0.004, Epoch time = 6.972s, All time = 7.413s
train
eval
bleu
Epoch: 3, Train loss: 0.284, Val loss: 0.064, Blue: 0.004, Epoch time = 8.292s, All time = 8.999s
train
eval
bleu
Epoch: 4, Train loss: 0.283, Val loss: 0.063, Blue: 0.005, Epoch time = 8.259s, All time = 8.748s
train
eval
bleu
Epoch: 5, Train loss: 0.282, Val loss: 0.063, Blue: 0.005, Epoch time = 8.095s, All time = 8.557s
train
eval
bleu
Epoch: 6, Train loss: 0.282, Val loss: 0.063, Blue: 0.005, Epoch time = 8.817s, All time = 9.288s
train
eval
bleu
Epoch: 7, Train loss: 0.281, Val loss: 0.062, Blue: 0.006, Epoch time = 8.551s, All time = 9.033s
train
eval
bleu
Epoch: 8, Train loss: 0.280, Val loss: 0.062, Blue: 0.007, Epoch time = 7.903s, All time = 8.380s
train
eval
bleu
Epoch: 9, Train loss: 0.278, Val loss: 0.061, Blue: 0.004, Epoch time = 